In [145]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_file):
    text = ""
    pdf_reader = PdfReader(pdf_file)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Usage example
pdf_path = r"F:\LLM\Vedantcv (3).pdf"
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

VEDANT  SUNIL  VASAIKAR  
• +917770026534  • vedantvasaikar@gmail.com  • linkedin.com/in/ vedant -vasaikar -/     
  SUMMARY  
 
Passionate computer engineering student with a growing fascination for machine learning. Actively involved in hands -on projects to dive 
deeper into ML concepts, cultivating a keen interest in the field. Eager to delve deeper into the diverse r ealms of engineering and Enthusiastic 
about contributing creatively to cutting -edge projects that leverage a multidisciplinary approach. Ready to apply engineering skills and 
knowledge to innovate and solve complex problems across various domains.  
 
Experi ence  / Internships  
 Machine Learning Intern at NullClasses . 
 Data Analyst Intern at IBM.  
 Web -Development Intern at Oasis Infobyte . 
 
EDUCATION  
 
 BE, Computer  Engineering                  Pursuing                                                                                                                                                     

In [146]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_text(extracted_text)



In [147]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer(r'F:\LLM\Sentence_Transformers_model')
# embeddings = model.encode(chunks)


In [148]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hembeddings = HuggingFaceEmbeddings(model_name=r"F:\LLM\Sentence_Transformers_model")

In [149]:
vdb = FAISS.from_texts( texts= chunks, embedding=hembeddings)
print(vdb.index.ntotal)

9


In [150]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

llm = LlamaCpp(
    model_path=r"F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:         

In [151]:

# Building the retriever
retriever = vdb.as_retriever(search_kwargs={'k': 3})

In [152]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


template = """

You are a information retrieval AI. Format the retrieved information as a table or text


Use only the context for your answers, do not make up information

query: {query}

{context} 
"""

# Converts the prompt into a prompt template
prompt = ChatPromptTemplate.from_template(template)

model = LlamaCpp(
    model_path=r"F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    temperature=0.4,
    max_tokens=2000,
    top_p=1,
    verbose=True,  # Verbose is required to pass to the callback manager
)

# Construction of the chain
chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "query":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | model | StrOutputParser()
 
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:         

llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                      tokenizer.ggml.merges arr[str,61249]   = ["▁ t", "e r", "i n", "▁ a", "e n...
llama_model_loader: - kv  17:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  18:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  19:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  20:            tokenizer.ggml.padding_token_id u32              = 2
llama_model_loader: - kv  21:                    tokenizer.chat_template str              = {% f

In [153]:
prompt =""" who is vedant
"""

print(chain.invoke(prompt))


llama_print_timings:        load time =     429.23 ms
llama_print_timings:      sample time =      33.19 ms /    58 runs   (    0.57 ms per token,  1747.46 tokens per second)
llama_print_timings: prompt eval time =   10785.92 ms /   424 tokens (   25.44 ms per token,    39.31 tokens per second)
llama_print_timings:        eval time =    3138.23 ms /    57 runs   (   55.06 ms per token,    18.16 tokens per second)
llama_print_timings:       total time =   14038.97 ms /   481 tokens



query: What is the percentage of Pace JR College Andheri in terms of SSC students who passed May 2019 and what is the percentage of St. Anthony's Convent High School in terms of TECHNICAL SKILLS?
